# Descarga de datos

## URL de las distintas fuentes de datos

**PRODUCCIÓN ANUAL AGRÍCOLA**
- https://nube.siap.gob.mx/cierreagricola/

**PRODUCCIÓN MENSUAL AGRÍCOLA**
- https://nube.siap.gob.mx/avance_agricola/
- http://infosiap.siap.gob.mx:8080/agricola_siap_gobmx/ResumenProducto.do

**INTENCION DE SIEMBRA**
- https://nube.siap.gob.mx/agroprograma/

**SISTEMA DE INFORMACIÓN AGROALIMENTARIA DE CONSULTA**
- http://infosiap.siap.gob.mx/gobmx/datosAbiertos_a.php

**ESTADÍSTICA DE USO TECNOLÓGICO Y DE SERVICIOS EN LA SUPERFICIE AGRÍCOLA**
- Fertilizada: https://nube.siap.gob.mx/index.php/s/kTLGLipCBEBiEFM/download
- Semilla mejorada y semilla criolla: https://nube.siap.gob.mx/index.php/s/OjKcZZL8QClaK6U/download
- Mecanizada: https://nube.siap.gob.mx/index.php/s/P7yK8NWmppStv7i/download
- Riego: https://nube.siap.gob.mx/index.php/s/cz6nETNW4b0krzD/download
- Sanidad: https://nube.siap.gob.mx/index.php/s/0l0Vw6VUf8fQ0dI/download
- Asistencia técnica: https://nube.siap.gob.mx/index.php/s/EHaUuYSEnHqHLqA/download

**INFORMACIÓN AGRÍCOLA DE GÉNERO**
- 2015 - 2017: https://www.gob.mx/cms/uploads/attachment/file/399835/Estadistica_de_G_nero_2017.pdf
- 2016 - 2018: https://www.gob.mx/cms/uploads/attachment/file/515428/Participaci_n_de_mujeres_en_valor_2016_17_18.pdf
- 2017 - 2019: https://www.gob.mx/cms/uploads/attachment/file/630415/Estadistica_de_Genero_2019.pdf

**MONOGRAFÍAS DE CULTIVOS**
- https://www.gob.mx/siap/documentos/monografias



## Librerias

In [1]:
# !pip install lxml
# !pip install pandas

In [2]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import pandas as pd
import warnings
import requests
     

## Catalogos para API: JSON de Victor

Nota: sí vienen los 64 cultivos que aparecen en la página

In [3]:
if not os.path.exists('catalogos_api'):
    os.mkdir("catalogos_api")

df = pd.read_json('Victor-dict.json',orient='index')

tag = ["municipio","anioagric","cicloProd","modalidad","entidad","cultivo","distrito"]
name_csv = ['Municipio','Anio_Agricola','Ciclo_Prod','Modalidad','Entidad','Cultivo','Distrito']

for i in range(7):
    # Toma los datos pertenecientes al tag
    df_aux2 = df.loc[df.index.str.startswith(tag[i])]
    df_aux = df_aux2.copy()
    # Elimina la str del tag+_ de la columna index
    df_aux.index = df_aux.index.str.replace(tag[i]+'_', "")
    # Convierte el indice en columna
    df_aux.reset_index(inplace=True)
    # Cambia los nombres de las columnas
    df_aux.rename(columns={0: "ID","index":"Name"}, inplace=True)
    # Crea el path donde se guardara
    ruta_completa = os.path.join("catalogos_api",name_csv[i])
    # Guardar el DataFrame en el archivo CSV en la carpeta "catalogos_api"
    df_aux.to_csv(ruta_completa + '.csv', index=False)

#Elimina los df
del df_aux, df_aux2

In [4]:
# df = pd.read_csv('./catalogos_api/anioagric.csv', index_col=False)


## Filtrar Municipios de Sonora

In [5]:
# Define el enlace compartido del archivo en Google Drive
url = 'https://sistemadatos.sonora.gob.mx/OpenDataFiles/1573/2023050512492324-file.xlsx'

nombre_archivo = "catalogos.xlsx"  # Puedes ajustar el nombre según tu preferencia

# Realiza la solicitud GET para descargar el archivo
response = requests.get(url)

# Verifica si la descarga fue exitosa (código de respuesta 200)
if response.status_code == 200:
    # Abre un archivo local y guarda el contenido descargado
    with open(nombre_archivo, 'wb') as file:
        file.write(response.content)
    print(f"El archivo '{nombre_archivo}' se ha descargado correctamente.")
else:
    print("Error al descargar el archivo.")

El archivo 'catalogos.xlsx' se ha descargado correctamente.


In [11]:
df = pd.read_excel('./catalogos.xlsx',sheet_name='Municipios')
df = df[['MUNICIPIOS','MUNI']]
df['Municipios'] = df['MUNICIPIOS'].str.lower()
df

,MUNICIPIOS,MUNI,Municipios
0,GENERAL PLUTARCO ELÍAS CALLES,70,general plutarco elías calles
1,ALTAR,4,altar
2,CANANEA,19,cananea
3,CABORCA,17,caborca
4,ATIL,7,atil
...,...,...,...
67,QUIRIEGO,49,quiriego
68,ETCHOJOA,26,etchojoa
69,NAVOJOA,42,navojoa
70,HUATABAMPO,33,huatabampo


In [12]:
df2 = pd.read_csv('./catalogos_api/Municipio.csv')
df2['Name_min'] = df2['Name'].str.lower()
df2

,Name,ID,Name_min
0,Todos,0,todos
1,Aguascalientes,1,aguascalientes
2,Asientos,2,asientos
3,Calvillo,3,calvillo
4,Cosío,4,cosío
...,...,...,...
2322,Villa de Cos,51,villa de cos
2323,Villa García,52,villa garcía
2324,Villa González Ortega,53,villa gonzález ortega
2325,Villanueva,55,villanueva


In [13]:
# Filtra las filas de df2 que coinciden con los municipios de df
df_filtrado = df2[df2['Name_min'].isin(df['Municipios'])]


In [14]:
df_filtrado

,Name,ID,Name_min
167,Rayón,50,rayón
254,Rosario,51,rosario
268,Benito Juárez,4,benito juárez
573,Magdalena,98,magdalena
630,Villa Hidalgo,54,villa hidalgo
...,...,...,...
1878,Trincheras,64,trincheras
1879,Tubutama,65,tubutama
1880,Ures,66,ures
1881,Villa Pesqueira,68,villa pesqueira


In [32]:
df2[df2['ID'] == 26]

,Name,ID,Name_min
107,Chenalhó,26,chenalhó
229,Gran Morelos,26,gran morelos
304,San Dimas,26,san dimas
338,Romita,26,romita
387,Cuetzala del Progreso,26,cuetzala del progreso
462,Huazalingo,26,huazalingo
545,Concepción de Buenos Aires,26,concepción de buenos aires
660,Chapa de Mota,26,chapa de mota
775,Chinicuila,26,chinicuila
888,Tlayacapan,26,tlayacapan


## Datos mensuales

In [14]:
# Creacion del folder raw
if not os.path.exists('raw'):
    os.mkdir("raw")

In [17]:
# Comando para ignorar warnings (salen en cada request)
warnings.filterwarnings('ignore')

In [48]:
anio = pd.read_csv('./catalogos_api/Anio_Agricola.csv').ID.to_list()
ciclo_prod = pd.read_csv('./catalogos_api/Ciclo_Prod.csv').ID.to_list()
cultivo = pd.read_csv('./catalogos_api/Cultivo.csv').ID.to_list()
distrito = pd.read_csv('./catalogos_api/Distrito.csv').ID.to_list()
entidad = 26 # Sonora: 26
modalidad = pd.read_csv('./catalogos_api/Modalidad.csv').ID.to_list()
municipio = pd.read_csv('./catalogos_api/Municipio.csv').ID.to_list()

In [49]:
cultivo.remove(0)
distrito.remove(0)
municipio.remove(0)

In [50]:
print(anio)
print(ciclo_prod)
print(cultivo)
print(distrito)
print(entidad)
print(modalidad)
print(municipio)

[2023, 2022, 2021, 2020, 2019, 2018]
[1, 2, 3, 4, 5]
[6, 7, 8, 9, 15, 17, 27, 36, 42, 43, 53, 57, 60, 61, 63, 70, 74, 77, 78, 89, 105, 110, 112, 122, 128, 132, 149, 150, 152, 159, 163, 168, 175, 206, 212, 222, 224, 225, 230, 234, 241, 259, 268, 272, 283, 285, 289, 290, 296, 304, 320, 326, 373, 374, 375, 379, 389, 390, 392, 395, 400, 401, 416, 420]
[1, 2, 3, 5, 6, 7, 4, 9, 10, 11, 8, 12, 14, 192, 13, 15, 16, 17, 20, 24, 23, 22, 19, 27, 25, 26, 18, 21, 36, 38, 29, 28, 35, 33, 40, 30, 32, 34, 39, 41, 37, 31, -1, 42, 43, 44, 45, 191, 47, 46, 51, 52, 48, 50, 49, 55, 53, 56, 58, 54, 57, 59, 64, 63, 62, 61, 60, 67, 71, 72, 69, 70, 66, 68, 65, 77, 78, 80, 76, 75, 73, 79, 74, 82, 85, 81, 84, 92, 89, 83, 91, 90, 88, 86, 87, 93, 94, 98, 97, 96, 95, 99, 100, 101, 103, 102, 109, 106, 104, 107, 108, 110, 105, 115, 111, 116, 114, 117, 118, 113, 112, 120, 119, 122, 121, 125, 123, 124, 130, 131, 132, 128, 129, 127, 126, 136, 135, 134, 137, 133, 138, 141, 139, 148, 147, 144, 140, 145, 143, 149, 146, 193

In [82]:

# Endpoint
url = "https://nube.siap.gob.mx/avance_agricola/"

payload = {'xajax': 'reporte', # Para obtener la tabla
'xajaxr': '1696449941927', # Timestamp UNIX
'xajaxargs[]': [
    '1', # 1: Desglose por estados, 2: Cultivo total
    str(2022), # Anio
    '5', # ID Ciclo
    '3', # ID Modalidad
    '26', # ID Estado (0: Nacional)
    '--',
    '--',
    '7', # ID Cultivo
    '200201',
    '0',
    '0', # 1: Por municipio
    'undefined',
    'undefined',
    'undefined',
    str(4) # Valor del mes: va de 1 a 8, con 1 siendo Enero y 8 Agosto
    ]
}

headers = {
  'Cookie': 'PHPSESSID=45ri2k73cbp2iptcrufu88p360'
}

# Hacemos el request
response = requests.request("POST", url, headers=headers, data=payload, verify=False)
response.encoding='ISO-8859-1'



In [83]:
response.text

'<?xml version="1.0" encoding="iso-8859-1" ?><xjx><cmd n="as" t="Resultado" p="innerHTML"><![CDATA[\r\n\t\t\t<div class="titulosTabla">\r\n\t\t\t\t<p>\r\n\t\t\t\t\t<strong>A&ntilde;o agr&iacute;cola:</strong> 2022<br />\r\n\t\t\t\t\t<strong>Producci&oacute;n Agr&iacute;cola</strong><br />\r\n\t\t\t\t\t<strong>Ciclo:</strong> Ciclicos - Perennes<br />\r\n\t\t\t\t\t<strong>Modalidad:</strong> Riego + Temporal<br />\r\n\t\t\t\t\t<strong>Cultivo:</strong> Aguacate (ton)     <br /><strong>Entidad Federativa:</strong> Sonora<br />\r\n\t\t\t\t\tSituaci&oacute;n al 30 de abril de 2022<br />\r\n\t\t\t\t</p>\r\n\t\t\t</div>\r\n\t\t\t\r\n\t\t\t<table id="Resultados-reporte" class="table table-striped table-responsive table-bordered " border="1">\r\n\t\t\t\t<tr class="estiloResult">\r\n\t\t\t\t\t<th rowspan="2">&nbsp;</th>\r\n\t\t\t\t\t<th rowspan="2">Entidad</th>\r\n\t\t\t\t\t<th rowspan="2">Distrito</th>\r\n\t\t\t\t\t<th colspan="3">Superficie<br />(ha)</th>\r\n\t\t\t\t\t<th rowspan="2">Producci